## 数据读取及预处理

In [1]:
import os
import mxnet as mx
from mxnet import gluon, autograd, init
from mxnet.gluon import data as gdata, loss as gloss, nn 
from mxnet.gluon.data.vision import transforms

transform_train = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),])
#     gdata.vision.transforms.Normalize([0.4914, 0.4822, 0.4465], 
#                                       [0.2023, 0.1994, 0.2010])])

train_ds = gdata.vision.ImageFolderDataset(root = r'.\images',
                                           flag=1)  # 0转换为灰度图，1转换为彩色图

loader = gluon.data.DataLoader  # 可迭代对象class，非迭代器class

train_data = loader(dataset = train_ds.transform_first(transform_train), 
                    batch_size = 128,
                    shuffle=True,
                    last_batch='keep')

C:\software\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 网络定义及初始化方法

In [2]:
from mxnet import nd
from mxnet.gluon import nn

 
class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                                  strides=strides)
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                       strides=strides)

    def hybrid_forward(self, F, x):
        """
        conv(3*3 kernel, unknow stride)->bn->relu->conv(3*3 kernel, 1*1 stride)-bn
        conv(1*1 kernal, unknow stride)
        """
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)
 
 
class ResNet(nn.HybridBlock):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = self.net = nn.HybridSequential()
            # 模块1
            net.add(nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            # 模块2
            for _ in range(3):
                net.add(Residual(channels=32))
            # 模块3
            net.add(Residual(channels=64, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=64))
            # 模块4
            net.add(Residual(channels=128, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=128))
            # 模块5
            net.add(nn.AvgPool2D(pool_size=8))
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))
 
    def hybrid_forward(self, F, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s'%(i+1, out.shape))
        return out
 
 
def get_net(ctx):
    num_outputs = 5
    net = ResNet(num_outputs)
    net.initialize(ctx=ctx, init=mx.init.Xavier())
    return net

## 预训练网络获取

In [3]:
from mxnet.gluon.model_zoo import vision as models

pretrained_net = models.resnet18_v2(pretrained=True)
finetune_net = models.resnet18_v2(classes=5)
finetune_net.features = pretrained_net.features
pretrained_net.features.collect_params().setattr('grad_req', 'null')  # 固定特征层参数
finetune_net.output.initialize(init.Xavier())

## 训练策略

In [4]:
import datetime
import sys
sys.path.append('..')
import gluonbook as gb
 
def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period, lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9, 'wd': wd})
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0.0
        train_acc = 0.0
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            label = label.astype('float32').as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
            train_acc += gb.accuracy(output, label)
            # print(gb.accuracy(output, label))
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_acc = gb.evaluate_accuracy(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Loss: %f, Train acc %f, Valid acc %f, "
                         % (epoch, train_loss / len(train_data),
                            train_acc / len(train_data), valid_acc))
        else:
            epoch_str = ("Epoch %d. Loss: %f, Train acc %f, "
                         % (epoch, train_loss / len(train_data),
                            train_acc / len(train_data)))
        prev_time = cur_time
        # net.export('./model/astro')
        net.save_parameters('./model/astro_{}'.format(epoch))
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

## 实际训练

In [6]:
ctx = gb.try_gpu()
num_epochs = 5
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 80
lr_decay = 0.1
batch_size = 32


finetune = False  # <-----是否使用yu'xun'lian
if finetune:
    finetune_net.collect_params().reset_ctx(ctx)
    finetune_net.hybridize()
    net = finetune_net
else:
    net = get_net(ctx)
    net.hybridize()
    net.load_parameters('./model/astro_0')

train(net, train_data, None, num_epochs, learning_rate,
      weight_decay, ctx, lr_period, lr_decay)

0.7734375
0.6796875
0.734375
0.7265625
0.734375
0.6875
0.796875
0.7578125
0.7734375
0.7734375
0.765625
0.7578125
0.7734375
0.765625
0.78125
0.828125
0.8203125
0.7578125
0.78125
0.7734375
0.8203125
0.8125
0.71875
0.8515625
0.765625
0.84375
0.8359375
0.7734375
0.796875
0.8515625
0.8
Epoch 0. Loss: 0.462500, Train acc 0.777823, Time 00:01:41, lr 0.01
0.78125
0.8046875
0.734375
0.796875
0.8046875
0.78125
0.8125
0.7734375
0.859375
0.7890625
0.8125
0.765625
0.828125
0.71875
0.828125
0.734375
0.7109375
0.8046875
0.8671875
0.8046875
0.859375
0.7890625
0.8359375
0.7421875
0.8046875
0.8125
0.734375
0.7421875
0.84375
0.8359375
0.8
Epoch 1. Loss: 0.438689, Train acc 0.793952, Time 00:01:47, lr 0.01
0.78125
0.84375
0.828125
0.84375
0.8515625
0.890625
0.890625
0.875
0.8515625
0.8359375
0.859375
0.8828125
0.8515625
0.8984375
0.875
0.8828125
0.875
0.8984375
0.9375
0.8828125
0.8125
0.84375
0.828125
0.828125
0.8046875
0.8203125
0.8125
0.8671875
0.8125
0.7421875
0.7
Epoch 2. Loss: 0.346429, Train acc 0.8